Import Part

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy import stats
from pandas import DataFrame

Const

In [2]:
FILE_NAME = 'vegas.csv'

Functions

In [3]:
def remove_outliers(df: DataFrame):
    Q1 = np.percentile(df['aqi'], 25,
                   interpolation = 'midpoint')
 
    Q3 = np.percentile(df['aqi'], 75,
                    interpolation = 'midpoint')
    IQR = Q3 - Q1
    ''' Removing the Outliers '''
    df = df[(df['aqi'] > (Q1 - 1.5 * IQR)) & (df['aqi'] < (Q3 + 1.5 * IQR))]
    return df

Loading dataset

In [4]:
df = pd.read_csv(os.path.join(FILE_NAME), header=0, index_col=0)
df.index = pd.to_datetime(df.index)
df = df.sort_values('date_local', ascending=True)

Removing outliers

In [5]:
df = remove_outliers(df)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Sliding window

In [7]:
for step in range(1, 50):
    values = DataFrame(df.values)
    dataframe = DataFrame()
    dataframe = pd.concat([values.shift(i) for i in range(step, -1, -1)], axis=1)
    columns = []
    for i in range(step, -1, -1):
        t = f"t-{i}" if i != 0 else "t"
        columns.extend([f"{t}-x{j}" for j in range(1, len(df.columns) + 1)])

    columns = columns[:-1] + ["y"]
    dataframe.columns = columns 
    dataframe = dataframe.dropna()

    X = dataframe.values[:, :-1]
    y = dataframe.values[:, -1]

    train_max_index = int(len(X) * 0.8)
    Xtrain = X[: train_max_index]
    ytrain = y[: train_max_index]
    Xtest = X[train_max_index:]
    ytest= y[train_max_index:]

    mdl = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=23)
    mdl.fit(Xtrain, ytrain)

    p = mdl.predict(Xtest)

    error = mean_squared_error(ytest, p)
    print('Lag size: %d, RMSE %.5f' % (step, error))

Lag size: 1, RMSE 70.47443
Lag size: 2, RMSE 68.77820
Lag size: 3, RMSE 68.89981
Lag size: 4, RMSE 70.60646
Lag size: 5, RMSE 69.71087
Lag size: 6, RMSE 70.16673
Lag size: 7, RMSE 70.34913
Lag size: 8, RMSE 70.55054
Lag size: 9, RMSE 70.55952
Lag size: 10, RMSE 70.73288
Lag size: 11, RMSE 71.13006
Lag size: 12, RMSE 71.20523
Lag size: 13, RMSE 71.68342
Lag size: 14, RMSE 71.24856
Lag size: 15, RMSE 71.56819
Lag size: 16, RMSE 72.22479
Lag size: 17, RMSE 72.10910
Lag size: 18, RMSE 72.00070
Lag size: 19, RMSE 73.03162
Lag size: 20, RMSE 72.56560
Lag size: 21, RMSE 72.76851
Lag size: 22, RMSE 72.85962
Lag size: 23, RMSE 72.52174
Lag size: 24, RMSE 72.93226
Lag size: 25, RMSE 72.86338
Lag size: 26, RMSE 72.53363
Lag size: 27, RMSE 73.25046
Lag size: 28, RMSE 73.06414
Lag size: 29, RMSE 73.52560
Lag size: 30, RMSE 73.61654
Lag size: 31, RMSE 73.41039
Lag size: 32, RMSE 74.26046
Lag size: 33, RMSE 74.22369
Lag size: 34, RMSE 74.06352
Lag size: 35, RMSE 74.68396
Lag size: 36, RMSE 74.24733
L